In [1]:
from sixopy.tw import Tw
from sixopy.gr import Gr
from sixopy.fb import Fb
from sixopy.bq import bQ
from google.cloud import bigquery
import numpy as np
import pandas as pd
import os

In [2]:
m = pd.read_json('fb_posts.json',orient='records',dtype=False)
m['created_date'] = m['created_time'].dt.strftime('%Y-%m-%d')
# m['created_date'] = m['created_at'].dt.strftime('%Y-%m-%d')

In [3]:
m.head()
m.dtypes

anger_reactions                                    int64
caption                                           object
comments                                           int64
comments_insights                                  int64
created_time                              datetime64[ns]
description                                       object
haha_reactions                                     int64
hide_all_clicks                                    int64
hide_clicks                                        int64
id                                                object
lifetime_average_time_video_viewed                 int64
lifetime_engaged_users                             int64
lifetime_organic_watches_at_95_percent             int64
lifetime_paid_watches_at_95_percent                int64
lifetime_post_consumers                            int64
lifetime_post_consumptions                         int64
lifetime_post_organic_impressions                  int64
lifetime_post_organic_reach    

In [5]:
creds = r"C:\Users\kkhitalishvili\Desktop\sixopy2\sixopy\sixopy-test\Denmark House Sales-c7bf498eeaeb.json"
# creds = "Denmark House Sales-c7bf498eeaeb.json"
bq = bQ(creds)

# bq.create_dataset('social_data')

s = pd.io.json.build_table_schema(m,index=False)
for d in s['fields']:
    if d['type'] == 'number': d['type'] = 'float'
s = [bigquery.SchemaField(p['name'].replace('(','').replace(')',''),p['type']) for p in s['fields']]
s
table = 'sixers_facebook'
bq.delete_table('social_data',table)
bq.create_table('social_data',table,schema = s)

social_data.sixers_facebook is deleted.


In [6]:
m = m.replace({np.nan: None})
bq.stream_data('social_data',table,m.to_dict('records'))

Loaded 1 row into social_data:sixers_facebook
